### Generate SQL query string, then read Google BigQuery to access M-Lab data and get ISP name using mlabnetdb

Created by John Burt, for allTBD group.

This notebook is a demo showing how to generate a query based on Kinga's R based query script to access M-Lab data. 



In [1]:
import pandas as pd
import datetime
import calendar

# This function takes as input the metric, mlab_location, AS number, 
# start_time, end_time and the optional country (the default 
# country is set to US)
# Check out the MLabServers.csv file to look up possible values for the
# mlab_location and AS variables.  The mlab_location should be entered using 
# quotation marks, the AS should be entered as an integer.
# The choices for the metric are: "dtp", "rtt", and "prt" for download 
# throughput, round trip time and packet retransmission respectively
# The start_time, end_time info should be entered in the 'mm/dd/yy' format
# The output of the function, when successful, is a text file, called
# query.txt

def query_writer(metric, mlab_location, AS, start_time, end_time, country = 'US' ): 
      
    #DEFINING THE BASIC QUERIES FOR EACH METRIC

    #The basic query for download throughput
    dtp_basic_query = ("SELECT "
        "\nweb100_log_entry.log_time AS log_time, "
        "\nconnection_spec.client_geolocation.city  AS client_city, "
        "\nconnection_spec.client_geolocation.area_code As client_area_code, "
        "\nweb100_log_entry.connection_spec.remote_ip AS client_ip, "
        "\nweb100_log_entry.connection_spec.local_ip AS MLab_ip, "
        "\n8 * (web100_log_entry.snap.HCThruOctetsAcked / "
        "\n(web100_log_entry.snap.SndLimTimeRwin + "
        "\nweb100_log_entry.snap.SndLimTimeCwnd + "
        "\nweb100_log_entry.snap.SndLimTimeSnd)) AS download_Mbps "
        "\nFROM "
        "\n[plx.google:m_lab.ndt.all] "
        "\nWHERE "
        "\nIS_EXPLICITLY_DEFINED(web100_log_entry.connection_spec.remote_ip) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.connection_spec.local_ip) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.HCThruOctetsAcked) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SndLimTimeRwin) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SndLimTimeCwnd) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SndLimTimeSnd) "
        "\nAND project = 0 "
        "\nAND IS_EXPLICITLY_DEFINED(connection_spec.data_direction) "
        "\nAND connection_spec.data_direction = 1 "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.is_last_entry) "
        "\nAND web100_log_entry.is_last_entry = True "
        "\nAND web100_log_entry.snap.HCThruOctetsAcked >= 8192 "
        "\nAND (web100_log_entry.snap.SndLimTimeRwin + "
        "\nweb100_log_entry.snap.SndLimTimeCwnd + "
        "\nweb100_log_entry.snap.SndLimTimeSnd) >= 9000000 "
        "\nAND (web100_log_entry.snap.SndLimTimeRwin + "
        "\nweb100_log_entry.snap.SndLimTimeCwnd +   "
        "\nweb100_log_entry.snap.SndLimTimeSnd) < 3600000000 "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.CongSignals) "
        "\nAND web100_log_entry.snap.CongSignals > 0 "
        "\nAND (web100_log_entry.snap.State == 1 "
        "\nOR (web100_log_entry.snap.State >= 5 "
        "\nAND web100_log_entry.snap.State <= 11))")


    #The basic query for finding round trip time 
    rtt_basic_query = ("SELECT "
        "\nweb100_log_entry.log_time AS log_time, "
        "\nconnection_spec.client_geolocation.city  AS client_city, "
        "\nconnection_spec.client_geolocation.area_code As client_area_code, "
        "\nweb100_log_entry.connection_spec.remote_ip AS client_ip, "
        "\nweb100_log_entry.connection_spec.local_ip AS MLab_ip, "
        "\nweb100_log_entry.snap.MinRTT AS min_rtt "
        "\nFROM "
        "\n[plx.google:m_lab.ndt.all] "
        "\nWHERE "
        "\nIS_EXPLICITLY_DEFINED(web100_log_entry.connection_spec.remote_ip) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.connection_spec.local_ip) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.HCThruOctetsAcked) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SndLimTimeRwin) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SndLimTimeCwnd) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SndLimTimeSnd) "
        "\nAND project = 0 "
        "\nAND IS_EXPLICITLY_DEFINED(connection_spec.data_direction) "
        "\nAND connection_spec.data_direction = 1 "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.is_last_entry) "
        "\nAND web100_log_entry.is_last_entry = True "
        "\nAND web100_log_entry.snap.HCThruOctetsAcked >= 8192 "
        "\nAND (web100_log_entry.snap.SndLimTimeRwin + "
        "\nweb100_log_entry.snap.SndLimTimeCwnd + "
        "\nweb100_log_entry.snap.SndLimTimeSnd) >= 9000000 "
        "\nAND (web100_log_entry.snap.SndLimTimeRwin + "
        "\nweb100_log_entry.snap.SndLimTimeCwnd +   "
        "\nweb100_log_entry.snap.SndLimTimeSnd) < 3600000000 "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.MinRTT) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.CountRTT) "
        "\nAND web100_log_entry.snap.CountRTT > 10 "
        "\nAND (web100_log_entry.snap.State == 1 "
        "\nOR (web100_log_entry.snap.State >= 5 "
        "\nAND web100_log_entry.snap.State <= 11))")


    #The basic query for packet retransmission 
    prt_basic_query = ("SELECT "
        "\nweb100_log_entry.log_time AS log_time, "
        "\nconnection_spec.client_geolocation.city  AS client_city,  "
        "\nconnection_spec.client_geolocation.area_code As client_area_code,  "
        "\nweb100_log_entry.connection_spec.remote_ip AS client_ip, "
        "\nweb100_log_entry.connection_spec.local_ip AS MLab_ip, "
        "\n(web100_log_entry.snap.SegsRetrans / web100_log_entry.snap.DataSegsOut) AS packet_retransmission_rate "
        "\nFROM "
        "\n[plx.google:m_lab.ndt.all] "
        "\nWHERE "
        "\nIS_EXPLICITLY_DEFINED(web100_log_entry.connection_spec.remote_ip) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.connection_spec.local_ip) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.HCThruOctetsAcked) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SndLimTimeRwin) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SndLimTimeCwnd) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SndLimTimeSnd) "
        "\nAND project = 0 "
        "\nAND IS_EXPLICITLY_DEFINED(connection_spec.data_direction) "
        "\nAND connection_spec.data_direction = 1 "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.is_last_entry) "
        "\nAND web100_log_entry.is_last_entry = True "
        "\nAND web100_log_entry.snap.HCThruOctetsAcked >= 8192 "
        "\nAND (web100_log_entry.snap.SndLimTimeRwin + "
        "\nweb100_log_entry.snap.SndLimTimeCwnd + "
        "\nweb100_log_entry.snap.SndLimTimeSnd) >= 9000000 "
        "\nAND (web100_log_entry.snap.SndLimTimeRwin + "
        "\nweb100_log_entry.snap.SndLimTimeCwnd +   "
        "\nweb100_log_entry.snap.SndLimTimeSnd) < 3600000000 "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.SegsRetrans) "
        "\nAND IS_EXPLICITLY_DEFINED(web100_log_entry.snap.DataSegsOut) "
        "\nAND web100_log_entry.snap.DataSegsOut > 0 "
        "\nAND (web100_log_entry.snap.State == 1 "
        "\nOR (web100_log_entry.snap.State >= 5 "
        "\nAND web100_log_entry.snap.State <= 11))")
    
    #SELECTING THE RIGHT BASIC QUERY
    if metric == "dtp":
        basic = dtp_basic_query
    elif metric == "rtt" :
        basic = rtt_basic_query
    elif metric == "prt":
        basic = prt_basic_query
    else:
        print("The metric entered is invalid!")
        return

    #FINDING MLAB SERVER IPS
    servers = pd.read_csv('MLabServers.csv')
    #print(servers)
    cond = servers[(servers.City==mlab_location) & (servers.AS==AS) ]
    #print(cond)
    if cond.empty:
        print("There are no MLab servers satisfying the conditions entered.")
        return
    else:
       ips = cond.IP
    #print("\n",ips)

    #WRITING THE MLAB SERVER CONDITION
    mlab_serv_var = "web100_log_entry.connection_spec.local_ip"
    mlab_ips_cond = "\nAND ("
    for ip in ips[:-1]:
        mlab_ips_cond += mlab_serv_var + "=='" + ip + "' OR "
    mlab_ips_cond += mlab_serv_var + "=='" + str(ips[-1:].values[0]) + "')"
    #print(mlab_ips_cond)
    
    #CONVERTING DATE TO UNIX TIMESTAMP
    try:
        dt = datetime.datetime.strptime(start_time, "%m/%d/%y")
        start_time_unix = calendar.timegm(dt.timetuple())
    except:
        print("The start_time entered is invalid!")
        return
        
    try:
        dt = datetime.datetime.strptime(end_time, "%m/%d/%y")
        end_time_unix = calendar.timegm(dt.timetuple())
    except:
        print("The end_time entered is invalid!")
        return
    
    #WRITING THE TIME CONDITION
    tstamp_var = "web100_log_entry.log_time"
    tframe_cond = ("\nAND " + tstamp_var + "<=" + "%d"%(end_time_unix) +
        "\nAND " + tstamp_var + ">=" + "%d"%(start_time_unix))
    #print(tframe_cond)

    #WRITING THE COUNTRY CONDITION
    country_string = "'" + country + "'" 
    country_var = "connection_spec.client_geolocation.country_code"
    country_cond = "\nAND " + country_var + "==" + country_string
    #print(country_cond)

    #WRITING THE QUERY
    the_query = basic + country_cond + mlab_ips_cond + tframe_cond
    #with open("querypy.txt", "w") as text_file:
    #    text_file.write(the_query)

    return the_query

In [3]:
# test the query_writer output:
# the_query = query_writer("rtt", "New York", 174, "06/15/14", "05/13/15")
# print(the_query)

In [4]:
# function to acquire m-lab data into a pandas dataframe,   
#  then acquire and add ISP name and ASN as df columns

from pandas.io import gbq
import importlib
# add alltbd path so we can import mlabnetdb from there
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import mlabnetdb
#importlib.reload(mlabnetdb) 
from mlabnetdb import *


def acquire_mlab_data(project_id, metric, mlab_location, AS, start_time, end_time, country = 'US'):

    # generate the query
    querystring = query_writer(metric, mlab_location, AS, start_time, end_time, country)

    # read the query output into a pandas dataframe
    #   NOTE: the first time this runs, you will be prompted for an authorization key. 
    #    Click on the link provided, get the key string, paste it in, and go.
    df = gbq.read_gbq(querystring, project_id=project_id)

    # use mlabnetdb to get ISP names
    print("\ngetting ISP names.....")
    owner = []
    ispname = []
    asn = []
    for ip in df.client_ip:
        ipinfo = lookup(ip, date=None)
        if ipinfo:
            owner.append(ipinfo['autonomous_system_organization'])
            asn.append(ipinfo['autonomous_system_number'])
            ispname.append(ipinfo['isp'])
        else:
            print("error: for ip %s, ipinfo==None"%(ip))
            owner.append('')
            asn.append(0)
            ispname.append('')
    print("\n  DONE getting ISP names")

    # add IP_owner and IP_ASN columns to the dataframe
    df["IP_owner"] = owner
    df["IP_ASN"] = asn
    # get company name from owner string
    df["ISP_name"] = ispname
    
    return df

In [5]:
%%time

# this is my project ID, you will probably use a different one
project_id = 'mlab-194421'

# get the mlab data we'll use for plotting
df = acquire_mlab_data(project_id, "dtp", "New York", 174, "01/01/13", "01/01/14")

# show contents    
df.head()

Requesting query... ok.
Job ID: ba3cbdd3-c499-431a-8b23-1d75e088bfec
Query running...
  Elapsed 7.11 s. Waiting...
Query done.
Processed: 0.0 B Billed: 0.0 B
Standard price: $0.00 USD

Retrieving results...
Got 263021 rows.

Total time taken 35.17 s.
Finished at 2018-03-06 13:58:32.

getting ISP names.....
error: for ip 208.91.32.225, ipinfo==None
error: for ip 208.74.146.210, ipinfo==None
error: for ip 18.202.1.233, ipinfo==None
error: for ip 18.236.6.6, ipinfo==None
error: for ip 18.224.0.31, ipinfo==None

  DONE getting ISP names
Wall time: 1min 37s


In [36]:
datafile = "mlab_dtp_data_mlabnetdb.csv"
df.to_csv(datafile)

In [37]:
df.head()

,log_time,client_city,client_area_code,client_ip,MLab_ip,download_Mbps,IP_owner,IP_ASN,ISP_name
0,1362263294,Milton,302,71.200.161.150,38.106.70.147,5.164872,"Comcast Cable Communications, LLC",7922.0,Comcast Cable
1,1362205441,Sicklerville,856,76.117.120.10,38.106.70.160,14.599611,"Comcast Cable Communications, LLC",7922.0,Comcast Cable
2,1362248206,Watertown,315,184.153.192.24,38.106.70.147,8.638346,Time Warner Cable Internet LLC,11351.0,Time Warner Cable
3,1362196115,None,0,173.11.195.172,38.106.70.147,8.177175,"Comcast Cable Communications, LLC",7922.0,Comcast Business
4,1362186306,Glen Burnie,410,24.35.57.7,38.106.70.160,14.208418,WideOpenWest Finance LLC,12083.0,Broadstripe
